In [1]:
%matplotlib ipympl

import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
stocks = ['IPCALAB.NS', 'TATAMOTORS.NS', 'RELAXO.NS', 'PVR.NS']

start = dt.datetime(2019,1,1)
end = dt.datetime(2020,1,1)
data = web.get_data_yahoo(stocks, start, end)['Adj Close']

data.sort_index(inplace = True)

returns = data.pct_change()
returns.head()

Symbols,IPCALAB.NS,PVR.NS,RELAXO.NS,TATAMOTORS.NS
Date,,,,
2019-01-01,NaN,NaN,NaN,NaN
2019-01-02,-0.020212,-0.004421,0.000406,-0.028250
2019-01-03,-0.015253,-0.035810,0.010691,-0.014832
2019-01-04,-0.000825,0.027765,0.003950,0.029509
2019-01-07,-0.003685,0.006325,0.005201,0.025739


# Method of Monte Carlo Simulation

In [3]:
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()

risk_free_rate = 4.36/100

num_portfolios = 50000

#Set up an array to hold the results
res = np.zeros((3+len(stocks),num_portfolios))

for i in range(num_portfolios):
    weights = np.random.random(len(stocks))
    weights/=np.sum(weights)
    
    portfolio_return = np.sum(mean_daily_returns*weights)*252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) *np.sqrt(252)
    
    res[0,i] = portfolio_return
    res[1,i] = portfolio_volatility
    
    res[2,i] = ((res[0,i]-risk_free_rate))/res[1,i]
    
    for j in range(len(weights)):
        res[j+3,i] = weights[j]

returns_df = pd.DataFrame(res.T, columns = ['Returns', 'Volatility', 'Sharpe_Ratio',*[stocks[i] for i in range(len(stocks))]])
                                                                                   
#locating the position with maximum sharpe ratio
max_sharpe_ratio = returns_df.ix[returns_df['Sharpe_Ratio'].idxmax()]

#Locating the position with minimum volatility
min_vol_port = returns_df.ix[returns_df['Volatility'].idxmin()]

plt.scatter(returns_df.Volatility, returns_df.Returns, c = returns_df.Sharpe_Ratio, cmap = 'copper')
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.colorbar()

#Plot a red Star to highlight the potfolio with MAx Sharpe Ratio
plt.scatter(max_sharpe_ratio[1], max_sharpe_ratio[0], marker = (5,1,0), color = 'r', s = 1000)

#Plot a green star to highlight the Minimum Varience Portfolio
plt.scatter(min_vol_port[1], min_vol_port[0], marker = (5,1,0), color = 'g', s = 1000)
plt.show()

C:\Users\Utsav\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Utsav\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
print(max_sharpe_ratio)

Returns          0.474850
Volatility       0.181475
Sharpe_Ratio     2.376360
IPCALAB.NS       0.321460
TATAMOTORS.NS    0.094171
RELAXO.NS        0.581642
PVR.NS           0.002726
Name: 37536, dtype: float64


In [5]:
print(min_vol_port)

Returns          0.408182
Volatility       0.168072
Sharpe_Ratio     2.169201
IPCALAB.NS       0.364593
TATAMOTORS.NS    0.271364
RELAXO.NS        0.363818
PVR.NS           0.000225
Name: 4986, dtype: float64
